In [ ]:
# Collect dataset 

from datasets import load_dataset

dataset = load_dataset("wikipedia", "20200501.en")



In [ ]:
import tqdm
def hf_dump_chars_to_textfile(
                           file,
                           dataset,
                           data_keys,
                           max_char=-1):
  """Write part of a TFDS sentence dataset to lines in a text file.

  Args:
    dataset: tf.dataset containing string-data.
    data_keys: what keys in dataset to dump from.
    max_char: max character to dump to text file.

  Returns:
    name of temp file with dataset bytes, exact number of characters dumped.
  """
  line_count = 0
  with open(file, "a+") as outfp:
    char_count = 0
    for example in tqdm.tqdm(dataset):
      for k in data_keys:
        line = example[k] + "\n"
        char_count += len(line)
        line_count += 1
        outfp.write(line)
    if line_count % 100000 == 0:
        print("Done {} / {}".format(line_count, total_count)) # wiki count
  print("Total lines {}, chars {}".format(line_count, char_count))

total_count = 6078422
hf_dump_chars_to_textfile(
    "DATA/wikipedia.txt",
    dataset["train"],
    ("text",))

In [ ]:
### Train Sentencepiece tokenizer (Albert)

import sentencepiece as spm
spm.SentencePieceTrainer.train(input='DATA/wikipedia.txt',
                               model_prefix='bert-joint',
                               vocab_size=32000,
                               pad_id=0,
                               unk_id=1,
                               bos_id=-1,
                               user_defined_symbols=['(', ')', '"', '-', '.', '–', '£', '€'],
                               control_symbols=['[CLS]','[SEP]','[MASK]'],
                               shuffle_input_sentence=True,
                               input_sentence_size=10000000,
                               character_coverage=0.99995,
                               model_type='unigram')